In [8]:
import torch
print(torch.cuda.device_count())

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from unsloth import FastLanguageModel
import pprint
import json
from pathlib import Path
import transformers
import os
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
import re


1


In [2]:
device_num = 0
max_seq_length = 3000     # Unsloth auto supports RoPE Scaling internally!
dtype = None              # None for auto detection
load_in_4bit = False      # Use 4bit quantization to reduce memory usage. Can be False.
device = f"cuda:{device_num}"

if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")

Using eager for attention computation.


# LLM as Judge accuracy

In [9]:
judge_model_ids = [
    # 'MLP-KTLim/llama-3-Korean-Bllossom-8B',
    # "Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B",
    "deepseek-ai/DeepSeek-R1"
]

from huggingface_hub import snapshot_download

def model_dir(model_id):
    return f"/model/{model_id.replace('/', '-')}"

def get_judge_model(model_id):
    snapshot_download(repo_id=model_id, cache_dir=model_dir(model_id) + "/cache")
    
    pretrained_model = AutoModelForCausalLM.from_pretrained(
        model_id,  
        max_seq_length = max_seq_length,
        dtype = dtype,
        # load_in_4bit = load_in_4bit,
        # device_map=device,
        cache_dir=model_dir(model_id) + "/cache",
        attn_implementation=attn_implementation,
        trust_remote_code=True
        # local_files_only=True
    )

judge_models = {}
for model_id in judge_model_ids:
    judge_models[model_id] = get_judge_model(model_id)

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/e7/f7/e7f7b8810f2020d7ff50a46aef578773eecb7386ccba95924d21eae90685f990/1411532f8eab4dec6efe251cfdf83e8367221062b90c209bb2e1353f92d40b27?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00077-of-000163.safetensors%3B+filename%3D%22model-00077-of-000163.safetensors%22%3B&Expires=1739120763&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTEyMDc2M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2U3L2Y3L2U3ZjdiODgxMGYyMDIwZDdmZjUwYTQ2YWVmNTc4NzczZWVjYjczODZjY2JhOTU5MjRkMjFlYWU5MDY4NWY5OTAvMTQxMTUzMmY4ZWFiNGRlYzZlZmUyNTFjZmRmODNlODM2NzIyMTA2MmI5MGMyMDliYjJlMTM1M2Y5MmQ0MGIyNz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=nYHysD0JEScTt6FQ%7EXa7gpKS9pXWl-fn0bcIvRG4oWyXKfDjJlQkq7WwUuJpDr5W81P5r6SHC8as6exqumSprJLs7c2Hd695pzYKpgKsInoVsmLiRBW1QOx-NxFHLoVrgAO6DbZh%7Ei5Elmqa2zTTYPUrIpWTwz5Dw1rOhjR4qq%7EF8pjPKbRRJ8Xd0RMOoAyT5Ym9TgMqENfgdIW234Zj1WR9zji4qn

OSError: [Errno 28] No space left on device

In [ ]:
dataset_ts = json.load(open("training_dataset_v2_directsql_ts.json", "r"))
for i, data in enumerate(dataset_ts):
    dataset_ts[i]["Response"] = str(dataset_ts[i]["Response"])
print(dataset_ts[0])